In [4]:
import sys
import os
# setting path
sys.path.append('/work/scripts')
import pandas as pd
import seaborn as sns

In [5]:
import Sensitivity as sa
import output_utils as ou

# parameters influencing soil hydro/thermal state
### cmt_envground.txt
--------------------------------------------------------
#### tcsolid(m): Soil thermal conductivity for moss (W/mK)
current value: 0.25, range: 0.005-0.5

*Ekici et al. 2015,Jiang et al. 2015, O’Donnell et al. 2009*

#### tcsolid(f): Soil thermal conductivity for fibric (W/mK)
current value: 0.25, range: 0.005-0.5

*Ekici et al. 2015,Jiang et al. 2015, O’Donnell et al. 2009*

#### tcsolid(h): Soil thermal conductivity for humic (W/mK)
current value: 0.25, range: 0.02-2.0

*Ekici et al. 2015,Jiang et al. 2015, O’Donnell et al. 2009*

---------------------------------------------------------
#### porosity(m): porosity for moss layers (m3/m3)
current value: 0.98, range: 0.85-0.99

*O’Donnell et al. 2009*

#### porosity(f): porosity for fibric layers  (m3/m3)
current value: 0.95, range: 0.85-0.99

*O’Donnell et al. 2009*

#### porosity(h): porosity for humic layers  (m3/m3)
current value: 0.8, range: 0.7-0.9

*O’Donnell et al. 2009*

--------------------------------------------------------
#### bulkden(m): bulk density for moss (g/m3)
current value: 25,000, range: 10,000 - 80,000

*Tuomi et al. 2020, Rodionov et al. 2007, O’Donnell et al. 2009*

#### bulkden(f): bulk density for fibric (g/m3)
current value: 51,000, range: 20,000 - 200,000

*Tuomi et al. 2020, Rodionov et al. 2007, O’Donnell et al. 2009*

#### bulkden(h): bulk density for humic (g/m3)
current value: 176,000, range: 100,000 - 800,000

*Tuomi et al. 2020, Rodionov et al. 2007, O’Donnell et al. 2009*

--------------------------------------------------------
#### hksat(m): hydraulic conductivity at saturation for moss (mm/s)
current value: 0.15, range: 0.0002 - 30

*Ekici et al. 2015, Letts et al. 2000, Liu et al. 2019*

#### hksat(f): hydraulic conductivity at saturation for fibric (mm/s)
current value: 0.28, range: 0.0002 - 30

*Ekici et al. 2015, Letts et al. 2000, Liu et al. 2019*

#### hksat(h): hydraulic conductivity at saturation for humic (mm/s)
current value: 0.002, range: 0.00004 - 2.01

*Ekici et al. 2015, Letts et al. 2000, Liu et al. 2019*

---------------------------------------------------------
#### nfactor(s): Summer nfactor
current value: 1.5, range: 0.2-2.0

*Kade et al. 2006, Klene et al. 2001*

#### nfactor(w): Winter nfactor
current value: 1.0, range: 0.4-1.0

*Kade et al. 2006, Klene et al. 2001*

---------------------------------------------------------
#### snwalbmax
current value: 0.8, range: 0.7-0.85

*Te Beest et al. 2016, Petzold et al. 1975, Loranty et al. 2011*

#### snwalbmin
current value: 0.4, range: 0.4-0.6

*Te Beest et al. 2016, Petzold et al. 1975, Loranty et al. 2011*

---------------------------------------------------------
### cmt_dimground.txt
---------------------------------------------------------
#### snwdenmax (kg/m3)
current value: 250, range: 100-800

*Domine et al. 2016, Gerland et al. 1999, Muskett 2012*

#### snwdennew (kg/m3)
current value: 50, range: 10-250

*Domine et al. 2016, Gerland et al. 1999, Muskett 2012*


### cmt_bgcsoil.txt
---------------------------------------------------------
#### rhq10
current value: 2, range: 1.6-2.4
#### rhq10_w
current value: 2, range: 0.85-0.99

In [6]:


work_dir='/data/workflows/US-Prr_SWC_SA'
opt_run_setup='--tr-yrs=121 --sp-yrs=300 --eq-yrs=500 '

driver = sa.SensitivityDriver(work_dir = work_dir, clean=True)
driver.site = '/data/input-catalog/caribou-poker_merged/'
driver.opt_run_setup = opt_run_setup
driver.PXx ='1'
driver.PXy='0'

params = ['hksat(m)','hksat(f)','hksat(h)',
          'tcsolid(m)', 'tcsolid(f)', 'tcsolid(h)',
          'porosity(m)', 'porosity(f)', 'porosity(h)',
          'nfactor(s)', 'nfactor(w)',
          'rhq10']#, 'rhq10_w']
percent_diffs = [0.1, 0.1, 0.1,
                 0.1, 0.1, 0.1,
                 0.1, 0.1, 0.1,
                 0.1, 0.1,
                 0.1]#, 0.1]
bounds = [[0.001, 0.005], [1e-4, 0.05], [1e-4, 8e-4],
          [0.005, 0.5], [0.005, 0.5], [0.02, 2.0],
          [0.85, 0.99], [0.85, 0.99], [0.7, 0.9],
          [0.2, 2.0], [0.4, 1.0],
          [1.6, 2.4]]#, [1.6, 2.4]]
driver.logparams = [1, 1, 1,
                    1, 1, 1,
                    0, 0, 0,
                    0, 0,
                    0]#, 0]

driver.outputs = [
      { 'name': 'GPP', 'type': 'flux'},
      { 'name': 'RH','type': 'flux'},
      { 'name': 'LWCLAYER','type': 'layer'},
      { 'name': 'VWCLAYER','type': 'layer'},
      { 'name': 'TLAYER','type': 'layer'},
      { 'name': 'LAYERDEPTH','type': 'layer'},
      { 'name': 'LAYERDZ','type': 'layer'},
      { 'name': 'LAYERTYPE','type': 'layer'},
    ]

driver.design_experiment(Nsamples = 50, cmtnum = 13, params = params, percent_diffs = percent_diffs,
                         bounds=bounds, pftnums = [None]*len(params), sampling_method='uniform')


In [7]:
driver.setup_multi()

In [8]:
driver.sample_matrix

,hksat(m),hksat(f),hksat(h),tcsolid(m),tcsolid(f),tcsolid(h),porosity(m),porosity(f),porosity(h),nfactor(s),nfactor(w),rhq10
0,0.001312,0.000172,0.000303,0.008566,0.129852,0.027238,0.858132,0.971265,0.820223,1.474531,0.412351,2.375928
1,0.001566,0.002816,0.000111,0.378996,0.010094,0.632340,0.910472,0.890772,0.822371,0.451089,0.575287,1.893089
2,0.001330,0.003836,0.000201,0.090030,0.071047,0.052582,0.935056,0.873873,0.713010,1.907994,0.979379,2.246718
3,0.001153,0.010278,0.000132,0.023377,0.081734,1.250658,0.854814,0.977305,0.751756,1.392540,0.587027,2.016054
4,0.001214,0.001462,0.000114,0.009495,0.035256,0.051441,0.933706,0.979062,0.717699,0.552769,0.427136,1.860264
5,0.002099,0.000221,0.000783,0.193651,0.148545,0.048129,0.869729,0.962308,0.714910,1.976396,0.863347,1.758973
6,0.001394,0.000583,0.000195,0.086919,0.369576,0.023666,0.900185,0.866222,0.872621,1.321937,0.598539,1.650847
7,0.001797,0.000955,0.000539,0.058330,0.354901,0.175859,0.866743,0.949854,0.852157,1.210299,0.862580,1.995036
8,0.002248,0.005537,0.000170,0.306729,0.039870,0.269595,0.894010,0.921200,0.881513,0.648726,0.646230,2.204441
9,0.003038,0.003471,0.000413,0.188871,0.008423,0.027067,0.938677,0.972004,0.860734,0.535826,0.935535,2.031474


In [ ]:
driver.run_all_samples()

[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000008/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000000/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000002/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000010/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000006/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000004/config/config.js


[SA:run] /work/dvmdostem 